<h1> Document Vectorization </h1>

This experiment will be based on Google News pretrained Word2Vec and GloVe. 

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
import spacy
import en_core_web_md
import string 
import gensim.downloader
import math
from sklearn.preprocessing import StandardScaler

In [ ]:
NUM_OF_DOCUMENTS = 5
URL = "https://www.courtlistener.com/api/rest/v3/opinions/"

In [ ]:
def get_document(endpoint):
    r = requests.get(url = endpoint)
    data = r.json()
    verdict_text = data["plain_text"]
    verdict_text = verdict_text.replace("\n", " ")
    
    return verdict_text

<h2> Google News Word2Vec </h2>

In [3]:
word2vec = gensim.downloader.load('word2vec-google-news-300')

<h2> GloVe </h2>

In [4]:
glove = gensim.downloader.load('glove-wiki-gigaword-300')

[=====---------------------------------------------] 11.0% 41.2/376.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================--------------------------------] 36.1% 135.6/376.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[====================------------------------------] 40.4% 151.9/376.1MB downloaded

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==========================================--------] 85.0% 319.6/376.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[================================================--] 96.6% 363.3/376.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



<h3> Function for fitting training sets for average and sum of vectors </h3>

In [5]:
def fit_training(model):
    df_sum = pd.DataFrame()
    df_avg = pd.DataFrame()
    
    for idx, row in df.iterrows():
        document = row['document']
        words = document.split()
        word_vec = np.zeros((300, ))
        for word in words:
            if word.lower() in model.vocab:
                word_vec += model[word.lower()]
            elif word in model.vocab:
                word_vec += model[word]
    
        word_vec_avg = word_vec / len(words)
        word_vec = pd.Series(word_vec)
        df_sum = df_sum.append(pd.Series(word_vec), ignore_index=True)
        df_avg = df_avg.append(pd.Series(word_vec_avg), ignore_index=True)
    return df_sum, df_avg

In [6]:
def get_idx(type, n, document, model, avg):
    df_q = pd.DataFrame()
    words = document.split()
    word_vec = np.zeros((300, ))
    
    for word in words:
        if word.lower() in model.vocab:
            word_vec += model[word.lower()]
        elif word in model.vocab:
            word_vec += model[word]
            
    if avg == True:
        word_vec = word_vec/len(words)
        
    df_q = df_q.append(pd.Series(word_vec), ignore_index=True)

    strtype, asc, dftrain = getOtherVars(type, avg, model)

    distances = type(df_q, dftrain).flatten()
    indexes = np.argsort(distances)[::asc]
    indexes = indexes[:n]
    
    return indexes, strtype

<h3>  Inverse document frequency - IDF </h3>

In [7]:
idf = {}
for i in range(len(df.iloc[:, 0])):
    tokens = df.iloc[i, 0].translate(str.maketrans('', '', string.punctuation)).lstrip().rstrip().split()
    for w in tokens:
        try:
            idf[w.lower()].add(i)
        except:
            idf[w.lower()] = {i}

for key, value in idf.items():
    idf[key] = math.log(len(df.iloc[:, 0]) / (len(value)+1))

NameError: name 'df' is not defined

In [ ]:
def get_idx_idf(type, n, document, model, avg):
    df_q = pd.DataFrame()
    
    words = document.split()
    word_vec = np.zeros((300, ))
    for word in words:
        idfCoeff = 1
        if word.lower() in idf:
            idfCoeff = idf[word.lower()]
        if word.lower() in model.vocab:
            word_vec += model[word.lower()]*idfCoeff
        elif word in model.vocab:
            word_vec += model[word]*idfCoeff
            
    if avg == True:
        word_vec = word_vec / len(words)
        
    df_q = df_q.append(pd.Series(word_vec), ignore_index=True)

    strtype, asc, dftrain = getOtherVars(type, avg, model)

    distances = type(df_q, dftrain).flatten()
    indexes = np.argsort(distances)[::asc]
    indexes = indexes[:n]
    
    return indexes, strtype

<h3>  Part-of-speech - POS </h3>

In [ ]:
nlp = spacy.load("en_core_web_md")

In [ ]:
pos = {}

for sentence in df.iloc[:, 0]:
    for token in nlp(sentence.lower()):
        pos[token.text] = {token.pos_ : None}
        
for sentence in df_test.iloc[:, 1]:
    for token in nlp(sentence.lower()):
        pos[token.text] = {token.pos_ : None}

In [ ]:
for word, posTagAndVal in pos.items():
    
    for tag in posTagAndVal:
        if tag == "NOUN":
            posTagAndVal[tag] = 0.7
        elif tag == "PROPN": 
            posTagAndVal[tag] = 0.9
        elif tag == "ADJ":
            posTagAndVal[tag] = 0.8
        elif tag == "ADV":
            posTagAndVal[tag] = 0.7
        elif tag == "VERB":
            posTagAndVal[tag] = 0.6
        else:
            posTagAndVal[tag] = 0.4

In [ ]:
def get_idx_pos(type, n, document, model, avg):
    df_q = pd.DataFrame()
    
    words = document.split()
    word_vec = np.zeros((300, ))
    for word in words:
        posCoeff = 1
        if word.lower() in pos:
            posCoeff = next(iter(pos[word.lower()].values()))
        if word.lower() in model.vocab:
            word_vec += model[word.lower()]*posCoeff
        elif word in model.vocab:
            word_vec += model[word]*posCoeff
            
    if avg == True:
        word_vec = word_vec / len(words)
        
    df_q = df_q.append(pd.Series(word_vec), ignore_index=True)

    strtype, asc, dftrain = getOtherVars(type, avg, model)

    distances = type(df_q, dftrain).flatten()
    indexes = np.argsort(distances)[::asc]
    indexes = indexes[:n]
    
    return indexes, strtype

<h3>   Named Entity Recognition - NER </h3>

In [ ]:
ner = {}

for sentence in df.iloc[:, 0]:
    for token in nlp(sentence.lower()).ents:
        ner[token.text] = {token.label_: None}
        
for sentence in df_test.iloc[:, 1]:
    for token in nlp(sentence.lower()).ents:
        ner[token.text] = {token.label_: None}

In [ ]:
for word, nerTagAndVal in ner.items():
    
    for tag in nerTagAndVal:
        if (tag == "LOCATION") or (tag == "ORG") or (tag == "NORP") or (tag == "MONEY") or (tag == "WORK_OF_ART") or (tag == "LAW"):
            nerTagAndVal[tag] = 1.75
        elif (tag == "GPE") or (tag == "DATE") or (tag == "PERSON") or (tag == "FAC"): 
            nerTagAndVal[tag] = 1.5
        elif (tag == "ORDINAL") or (tag == "CARDINAL") or (tag == "PRODUCT") or (tag == "PERCENT") or (tag == "TIME"):
            nerTagAndVal[tag] = 1.2
        else:
            nerTagAndVal[tag] = 1

In [ ]:
def get_idx_ner(type, n, document, model, avg):
    df_q = pd.DataFrame()
    words = document.split()
    word_vec = np.zeros((300, ))
    for word in words:
        nerCoeff = 1
        if word.lower() in ner:
            nerCoeff = next(iter(ner[word.lower()].values()))
        if word.lower() in model.vocab:
            word_vec += model[word.lower()]*nerCoeff
        elif word in model.vocab:
            word_vec += model[word]*nerCoeff
            
    if avg == True:
        word_vec = word_vec / len(words)
        
    df_q = df_q.append(pd.Series(word_vec), ignore_index=True)

    strtype, asc, dftrain = getOtherVars(type, avg, model)

    distances = type(df_q, dftrain).flatten()
    indexes = np.argsort(distances)[::asc]
    indexes = indexes[:n]
    
    return indexes, strtype 

<h3> The Experiment </h3>

In [ ]:
wordEmbs = [word2vec, glove]
functions = [get_idx, get_idx_idf, get_idx_pos, get_idx_ner]
avgs = [True, False]
measures = [cosine_similarity]
